# 第四节 链（chain）,输出解析器与记忆(memory)

In [1]:
#导入语言模型
import os
from langchain_community.llms import Tongyi
from langchain_community.llms import SparkLLM
from langchain_community.llms import QianfanLLMEndpoint

import pandas as pd
#导入模版
from langchain.prompts import PromptTemplate

#导入聊天模型
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain_community.chat_models import ChatSparkLLM
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_community.chat_models import QianfanChatEndpoint

#输入三个模型各自的key

os.environ["DASHSCOPE_API_KEY"] = "sk-9bcefecc91cb4a20b04c5cabce8963dc"

os.environ["IFLYTEK_SPARK_APP_ID"] = ""
os.environ["IFLYTEK_SPARK_API_KEY"] = ""
os.environ["IFLYTEK_SPARK_API_SECRET"] = ""

os.environ["QIANFAN_AK"] = ""
os.environ["QIANFAN_SK"] = ""

In [2]:
model_ty = Tongyi(temperature=0.1)

## 链(chain)

只使用一个 LLM 去开发应用，比如聊天机器人是很简单的，但更多的时候，我们需要用到许多 LLM 去共同完成一个任务，这样原来的模式就不足以支撑这种复杂的应用。

为此 LangChain 提出了 Chain 这个概念，也就是一个所有组件的序列，能够把一个个独立的 LLM 链接成一个组件，从而可以完成更复杂的任务。举个例子，我们可以创建一个 chain，用于接收用户的输入，然后使用提示词模板将其格式化，最后将格式化的结果输出到一个 LLM。通过这种链式的组合，就可以构成更多更复杂的 chain。

在 LangChain 中有许多实现好的 chain，以最基础的 LLMChain 为例，它主要实现的就是接收一个提示词模板，然后对用户输入进行格式化，然后输入到一个 LLM，最终返回 LLM 的输出。

一般的应用：模版 + 模版 + 模版 ... + LLM模型

In [3]:
# 未使用链（chain） 单个一个模版 + 模型
template = """我希望你能充当新公司的命名顾问。一个生产{product}的公司好的中文名字是什么,请给出五个选项？"""

prompt = PromptTemplate(
    input_variables=["product"],
    template=template,
)

print(model_ty.invoke(prompt.format(product="彩色袜子")))

当然可以！以下是五个适合生产彩色袜子公司的中文名字选项：

1. **彩袜坊** - 简单直接，突出产品色彩丰富。
2. **缤纷步履** - 寓意每一步都充满色彩和活力。
3. **袜趣多** - 强调袜子的趣味性和多样性。
4. **七彩足韵** - 结合色彩与脚步的韵律之美。
5. **袜艺堂** - 体现袜子制作的艺术感和品质。

希望这些选项能为您的公司命名提供一些灵感！如果有其他需求或想要更具体的风格，欢迎告诉我。


In [4]:
# 使用链（chain）
from langchain.chains import LLMChain
 
prompt = PromptTemplate(
    input_variables=["product"],
    template="我希望你能充当新公司的命名顾问。一个生产{product}的公司好的中文名字是什么,请给出五个选项？",
)

# 创建一个链, 链的输入是模版，链的输出是模型
chain = LLMChain(llm=model_ty, prompt=prompt)

/var/folders/fq/3pmzyx3s60l0sjm0lfcpdfhh0000gn/T/ipykernel_5091/2359059665.py:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=model_ty, prompt=prompt)


In [5]:
chain.invoke("彩色袜子")

{'product': '彩色袜子',
 'text': '当然可以！以下是五个适合生产彩色袜子公司的中文名字选项：\n\n1. **彩袜坊** - 简洁明了，突出产品色彩丰富。\n2. **缤纷步履** - 寓意每一步都充满色彩和活力。\n3. **袜趣无限** - 强调袜子带来的乐趣和无限创意。\n4. **七彩丝途** - 结合色彩与丝滑质感，暗示舒适与时尚。\n5. **踏彩行** - 表达穿着彩色袜子行走时的优雅与自信。\n\n希望这些名字能为你的新公司提供一些灵感！如果有其他需求或想要更具体的风格，请随时告诉我。'}

In [6]:
type(chain.invoke("彩色袜子"))
# 返回的结果是一个字典
# 结果的key是"text"，值是模型的返回结果
print(chain.invoke("彩色袜子")["text"])

当然可以！以下是五个适合生产彩色袜子公司的中文名字建议：

1. **彩袜坊** - 简洁明了，直接体现了公司主营产品为彩色袜子。
2. **缤色袜语** - 结合“缤纷色彩”和“袜子”的概念，富有诗意且容易记住。
3. **袜趣多彩** - 强调袜子的趣味性和多样性，给人活泼的感觉。
4. **织彩袜梦** - 传达出通过袜子编织多彩梦想的品牌愿景，具有想象力。
5. **妙袜工坊** - 展现手工匠心与巧妙设计，同时突出袜子的独特性。

希望这些建议能激发你的灵感！如果需要进一步调整或有其他要求，请随时告诉我。


### 多个提示词

In [7]:
# 也可以使用generate，apply的生成结果简洁点
input_list = [
    {"product": "袜子"},
    {"product": "电脑"},
    {"product": "男鞋"}
]

chain.apply(input_list)

[{'text': '当然可以！以下是五个适合生产袜子公司的中文名字选项，每个名字都蕴含了独特的意义和品牌价值：\n\n1. **舒袜坊**  \n   - “舒”代表舒适、柔软，体现了袜子的穿着体验；“坊”则传递出手工与品质感。  \n\n2. **足悦堂**  \n   - “足”突出产品与脚部相关，“悦”表示愉悦、满意，寓意让双脚感到快乐和放松。  \n\n3. **步云轩**  \n   - “步云”象征轻盈如踏云端，给人一种行走时无比舒适的联想；“轩”增添高端氛围。  \n\n4. **丝暖家**  \n   - “丝”强调材质细腻，“暖”体现保暖功能，而“家”则拉近了与消费者的情感距离。  \n\n5. **锦足缘**  \n   - “锦”表示精致华美，“足”点明用途，“缘”寓意缘分，表达品牌希望与顾客建立长久联系的愿望。  \n\n如果你对某个名字感兴趣，我可以进一步帮你优化或扩展其品牌故事哦！'},
 {'text': '当然可以！以下是五个适合生产电脑公司的中文名字选项，每个名字都蕴含了独特的意义和品牌价值：\n\n1. **智核科技**  \n   - 寓意智慧与核心科技的结合，展现公司在计算机领域的专业性和创新能力。  \n\n2. **云极数字**  \n   - “云”代表云计算和未来趋势，“极”象征极致追求，体现公司在数字技术上的领先地位。  \n\n3. **星算未来**  \n   - 结合“星辰大海”的广阔愿景与计算能力，寓意公司致力于探索科技的无限可能。  \n\n4. **锐擎信息**  \n   - “锐”表示锐意进取，“擎”有驱动之意，整体传达出企业强大的动力和前瞻性。  \n\n5. **灵境互联**  \n   - 灵感来源于“虚拟现实”中的“灵境”，强调人机交互和智能连接的未来方向。  \n\n希望这些名字能为你的新公司提供灵感！如果需要进一步调整或解释，请随时告诉我。'},
 {'text': '当然可以！以下是五个适合生产男鞋公司的中文名字建议，每个名字都融合了品质、风格和品牌识别度的元素：\n\n1. **步云轩**  \n   - 寓意：行走如踏云端，展现舒适与优雅。  \n   - 适合强调高端商务或休闲男鞋的品牌。\n\n2. **锋尚履界**  \n   - 寓意：锋芒毕露，时尚前沿，“履”代表鞋子，整

### 多个变量

In [8]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="我希望你能充当新公司的命名顾问。公司新推出一款{product}产品，请帮给新产品取一个{style}中文名字，给出五个选项？",
)

chain_m = LLMChain(llm=model_ty, prompt=prompt) 

In [9]:
chain_m.invoke({"product": "混动越野汽车","style": "科技感"})

{'product': '混动越野汽车',
 'style': '科技感',
 'text': '当然可以！以下是五个具有科技感的中文名字选项，适用于混动越野汽车产品：\n\n1. **驰域X**  \n   （寓意驰骋疆域，结合未来科技感的“X”元素）\n\n2. **擎越**  \n   （“擎”代表动力与引擎，“越”象征跨越与超越）\n\n3. **电野**  \n   （体现电动混合动力与越野性能的融合）\n\n4. **征极者**  \n   （表达征服极限、探索未知的品牌精神）\n\n5. **凌动T**  \n   （“凌”展现霸气，“动”突出灵动与高效，T可作为技术或Terrain的缩写）\n\n如果需要进一步调整或增加其他风格的名字，请告诉我！'}

### 聊天模型的链


In [10]:
chat_ty = ChatTongyi()

human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template="我希望你能充当新公司的命名顾问。一个生产{product}的公司好的中文名字是什么,请给出五个选项？?",
            input_variables=["product"],
        )
    )
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chain_chat = LLMChain(llm=chat_ty, prompt=chat_prompt_template)

In [11]:
chain_chat.invoke({"product": "彩色袜子"})

{'product': '彩色袜子',
 'text': '当然可以！以下是五个适合生产彩色袜子公司的中文名字建议：\n\n1. **彩韵袜业**  \n   意味着多彩的韵味，突出袜子的颜色丰富和时尚感。\n\n2. **织色坊**  \n   “织”代表编织，“色”代表色彩，整体传达出精致与创意。\n\n3. **缤纷足履**  \n   强调袜子带来的缤纷体验，同时“足履”寓意行走时的美好陪伴。\n\n4. **七彩行者**  \n   既体现了袜子的多彩特性，又传递出行走自如的品牌理念。\n\n5. **妙袜工坊**  \n   简洁而有趣，让人联想到创意和趣味性，适合年轻化的品牌定位。\n\n希望这些建议能为你的公司提供灵感！如果需要进一步调整或扩展，请随时告诉我。'}

classwork 1

1, 创建一个多变量模版，要求给你的小孩取中文名字，变量分别为：父亲名字，母亲名字，小孩性别，名字的要求

2, 定义一个链，使用上面多变量模版，调用链，传入参数，输出结果

3，传入多组参数，通过apply方法，批量调用链，输出结果

4，定义一个包含聊天模型的链实现上述模版的结果输出



In [12]:
# 1. 定义一个多变量模版，要求给你的小孩取中文名字，变量分别为：父亲名字，母亲名字，小孩性别，名字的要求
prompt = PromptTemplate(
    input_variables=["children_name"],
    # "father_name","mother_name","gender","requirement"
    template="我希望你能给我的小孩起个{children_name}，父亲的名字是{father_name}，母亲的名字是{mother_name}，小孩的性别是{gender}，名字的要求是{requirement}，请给出五个选项？",
)

chain_name = LLMChain(
    llm=model_ty,
    prompt=prompt,
)

In [13]:
# 2. 定义一个链，使用上面多变量模版，调用链，传入参数，输出结果
name = chain_name.invoke({
    "children_name": "小名",
    "father_name": "徐福贵",
    "mother_name": "王家珍",
    "gender": "男",
    "requirement": "活的长久，聪明伶俐，事业有成",
})
print(name["text"])

根据您的要求，结合父母的名字以及希望孩子活的长久、聪明伶俐、事业有成的愿望，我为您提供了以下五个小名选项：

1. **福久** - “福”取自父亲名字“徐福贵”，寓意着孩子能够继承父亲的福气，活得长久且幸福。
2. **聪聪** - 寓意孩子聪明伶俐，思维敏捷，未来在学业和事业上都能取得优异成绩。
3. **家成** - “家”取自母亲名字“王家珍”，象征家庭美满，同时“成”字表达了对孩子事业有成的美好祝愿。
4. **长乐** - 寓意孩子长寿快乐，一生平安顺遂，保持乐观积极的心态面对生活中的各种挑战。
5. **瑞祥** - 表达了对孩子未来充满吉祥如意、事事顺利的期望，同时也暗含着聪明智慧之意。

每个名字都蕴含着美好的祝福与期许，希望其中有一个能符合您对孩子的期望。


In [14]:
# 3. 传入多组参数，通过apply方法，批量调用链，输出结果
#使用链（chain）
from langchain.chains import LLMChain
 
prompt = PromptTemplate(
    input_variables=["children_name"],
    template="我希望你能给我的小孩起个{children_name}，父亲的名字是徐福贵，母亲的名字是王家珍，小孩的性别是男性，名字的要求是能保佑活的长久，请给出2个选项？",
)

# 创建一个链, 链的输入是模版，链的输出是模型
chain_name_input = LLMChain(llm=model_ty, prompt=prompt) 

input_list = [
    {"children_name": "正式名"},
    {"children_name": "字"},
    {"children_name": "号"},
    {"children_name": "网名"},
    {"children_name": "英文名"},
]

chain_name_input.apply(input_list)

[{'text': '根据您提供的信息，父亲的名字是徐福贵，母亲的名字是王家珍，孩子的性别为男性，并且名字需要寓意长寿和健康。以下是两个正式名字的选项：\n\n### 1. **徐寿安**\n   - **寿**：直接寓意长寿，象征着孩子能够健康、长久地生活。\n   - **安**：代表平安、稳定，希望孩子一生无病无灾，安稳幸福。\n\n### 2. **徐永康**\n   - **永**：意为长久、永恒，表达对孩子长命百岁的祝福。\n   - **康**：象征健康、安康，寓意孩子身体强壮，一生顺遂。\n\n这两个名字都简洁大气，符合传统中国文化的美好祝愿，同时也易于书写和记忆。希望对您有所帮助！'},
 {'text': '为孩子取名字是一件非常重要的事情，尤其是希望名字能够保佑他活得长久、健康。根据您提供的信息，父亲名叫徐福贵，母亲名叫王家珍，孩子的性别是男性，以下是我为您精心挑选的两个名字选项：\n\n### 1. **徐长寿**\n   - **寓意**：名字直接表达了对生命长久的祝福，“长”象征时间的延续，“寿”则明确表示长寿、安康。\n   - **音韵**：简单易记，朗朗上口，传递出一种平和而深远的美好祝愿。\n\n### 2. **徐永安**\n   - **寓意**：“永”意味着永恒、长久；“安”代表平安、稳定。合在一起，寓意孩子一生平安顺遂，健康长寿。\n   - **音韵**：温和且充满力量，给人以安心之感。\n\n这两个名字都寄托了对孩子长寿健康的美好期望，同时结合了汉字的文化内涵与和谐的音韵美。希望这些建议能对您有所帮助！'},
 {'text': '根据你的要求，结合父母的名字以及希望孩子长寿的寓意，我为你提供了两个名字选项：\n\n### 选项一：徐长宁\n- **长**：象征长久、长寿，寓意生命绵延不绝。\n- **宁**：代表安宁、平静，寓意一生平安顺遂，没有大的波折。  \n整体寓意孩子能够健康长寿，生活安定祥和。\n\n---\n\n### 选项二：徐永康\n- **永**：意为永恒、长久，表达对生命持久的美好祝愿。\n- **康**：代表健康、安康，寓意身体强健，远离疾病。  \n整体寓意孩子能够长寿且身体健康，一生幸福无忧。\n\n这两个名字都带有吉祥和长寿的含义，同时简单易记，适合男性使用。希望你喜欢！'},
 {'te

In [15]:
# 4，定义一个包含聊天模型的链实现上述模版的结果输出

chat_ty = ChatTongyi()

# 
human_message_prompt = HumanMessagePromptTemplate(
    prompt = PromptTemplate(
        input_variables=["children_name"],
        template="我希望你能给我的小孩起个{children_name}，父亲的名字是徐福贵，母亲的名字是王家珍，小孩的性别是男性，名字的要求是能保佑活的长久，请给出2个选项？",
    )
)


chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])

chain_chat_name = LLMChain(llm=chat_ty, prompt=chat_prompt_template)

In [16]:
print(chain_chat_name.invoke({"children_name": "英文名"})["text"])

根据您提供的信息，父亲的名字是徐福贵，母亲的名字是王家珍，小孩是男性。考虑到名字需要有“保佑长寿”的寓意，我为您提供了以下两个英文名字选项：

1. **Everett**（埃弗雷特）：这个名字来源于古英语，意为“勇敢的战士”或“永远的力量”。它蕴含着持久和坚强的意义，寓意孩子能够健康长寿。

2. **Liam**（利亚姆）：这个名字源自爱尔兰语，是William的昵称，意为“坚定的保护者”。它简单而有力，象征着孩子能够得到守护并拥有长寿的人生。

这两个名字都具有积极的含义，并且在发音上也较为国际化，适合孩子的成长环境。希望这些建议对您有所帮助！


## 输出解析器（Output Parsers）

语言模型输出的是文本，但很多时候，我们可能希望获取比纯文本更结构化的信息，输出解析器（Output Parsers）就可以帮助我们获取结构化信息。输出解析器是 **结构化** 语言模型响应的类，其必须实现两个主要方法：

get_format_instructions() -> str：返回一个包含语言模型 *输出* 如何 *格式化* 的指令字符串。（指挥大语言模型 输入格式）

parse(str) -> Any：接受一个字符串（假设为语言模型的响应），并将其解析为某种结构。（回答结果 -》 解析为 python结构）


### 逗号分隔 列表 输出解析器CommaSeparatedListOutputParser

In [17]:
from langchain.output_parsers import CommaSeparatedListOutputParser

# 定义一个输出解析器，将模型的输出结果进行格式化
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

# 定义了一个 拥有定制化输出格式的模版
prompt = PromptTemplate(
    # {format_instructions} 是一个占位符，表示 输出格式 的说明
    template="请列出五个 {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)


In [18]:
# 给大模型说明，输出格式的类型
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [19]:
print(prompt)

input_variables=['subject'] input_types={} partial_variables={'format_instructions': 'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'} template='请列出五个 {subject}.\n{format_instructions}'


In [20]:
input0 = prompt.format(subject="中国著名作家")
output = model_ty.invoke(input0)
res0=output_parser.parse(output) 

In [21]:
# 未使用解析器，输出字符串
output

'鲁迅, 老舍, 巴金, 曹雪芹, 沈从文'

In [22]:
# 解析器的输出结果是一个python数据类型之一的 列表
res0

['鲁迅', '老舍', '巴金', '曹雪芹', '沈从文']

## 日期时间解析器DatetimeOutputParser

In [23]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.chains import LLMChain

# 定义一个输出解析器，将模型的输出结果进行 日期时间 格式化
output_parser = DatetimeOutputParser()

# 定义了一个 拥有定制化输出格式的模版
template = """Answer the users question:

{question}

{format_instructions}"""

prompt = PromptTemplate.from_template(template, partial_variables={"format_instructions": output_parser.get_format_instructions()})

# 创建一个链, 链的输入是模版，链的输出是模型
chain = LLMChain(prompt=prompt, llm=model_ty)

In [24]:
output = chain.invoke("中国建国是哪一年?")
output

{'question': '中国建国是哪一年?', 'text': '1949-10-01T00:00:00.000000Z'}

In [25]:
# 通过解析器 解析出 适用于python的日期时间
output_parser.parse(output["text"])

datetime.datetime(1949, 10, 1, 0, 0)

### 结构化输出解析器

一般是产生python类型中的 字典类型

In [26]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [27]:
# 定义字典的输出格式 字典的key（键值对）
response_schemas = [
    ResponseSchema(name="name", description="学生的姓名"),
    ResponseSchema(name="age", description="学生的年龄")
]
# 创建结构化输出解析器
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 定义了一个 拥有定制化输出格式的模版
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="回答下面问题,注意不要显示键的描述信息.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

# 输入模版化的提示词
_input = prompt.format_prompt(question="给我一个女孩的名字?")

output = model_ty(_input.to_string())
print(output)
print(output_parser.parse(output)) 

/var/folders/fq/3pmzyx3s60l0sjm0lfcpdfhh0000gn/T/ipykernel_5091/267767955.py:20: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = model_ty(_input.to_string())


```json
{
	"name": "小花",
	"age": "10"
}
```
{'name': '小花', 'age': '10'}


In [28]:
_input.to_string()

'回答下面问题,注意不要显示键的描述信息.\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"name": string  // 学生的姓名\n\t"age": string  // 学生的年龄\n}\n```\n给我一个女孩的名字?'

### classwork 2

1. classwork1中的取名应用，最后要求以列表的形式输出名字

2. 修改取名应用，要求使用结构化输出解析器，结构化输出名字和对应的意义

In [29]:
# 取名应用，最后要求以列表的形式输出名字
from langchain.output_parsers import CommaSeparatedListOutputParser

# 定义一个 逗号分隔 列表 输出解析器，将模型的输出结果进行格式化
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

# 定义一个多变量模版，要求给你的小孩取中文名字，变量分别为：父亲名字，母亲名字，小孩性别，名字的要求
prompt = PromptTemplate(
    input_variables=["children_name"],
    template=""" 
        我希望你能给我的小孩起个{children_name}，
        父亲的名字是{father_name}，
        母亲的名字是{mother_name}，
        小孩的性别是{gender}，
        名字的要求是{requirement}，
        请给出五个选项？
        /n{format_instructions}
    """,
    partial_variables={"format_instructions": format_instructions}
)

chain_name = LLMChain(
    llm=model_ty,
    prompt=prompt,
)
output = chain_name.invoke({
    "children_name": "小名",
    "father_name": "徐福贵",
    "mother_name": "王家珍",
    "gender": "男",
    "requirement": "活的长久，聪明伶俐，事业有成",
})
# 原始的大模型输出结果
res = output

res1 = output_parser.parse(res["text"])
# 解析器的输出结果是一个python数据类型之一的 列表
res1

['福宝', '福睿', '福成', '福聪', '福兴']

In [30]:
# 修改取名应用，要求 使用结构化 输出解析器 ，结构化输出名字和对应的意义
response_schemas = [
    ResponseSchema(name="name", description="小孩的名字"),
    ResponseSchema(name="meaning", description="名字的意义")
]
# 创建结构化输出解析器
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
# 定义了一个 拥有定制化输出格式的模版
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template=""" 
        回答下面问题,注意不要显示键的描述信息,
        \n{format_instructions}\n,
        父亲的名字是{father_name},
        母亲的名字是{mother_name},
        小孩的性别是{gender},
        请取个{requirement}的名字。
    """,
    input_variables=["father_name", "mother_name", "gender", "requirement"],
    partial_variables={"format_instructions": format_instructions}
)

chain_name0 = LLMChain(
    llm=model_ty,
    prompt=prompt,
)

res1 = chain_name0.invoke({
    "father_name": "徐福贵",
    "mother_name": "王家珍",
    "gender": "男",
    "requirement": "霸气",
})

res1
output_parser.parse(res1["text"])

{'name': '徐霸天', 'meaning': '霸气外露，显示无上的力量与权威'}

## Memory

大多数的 LLM 应用程序都会有一个会话接口，允许我们和 LLM 进行多轮的对话，并有一定的上下文记忆能力。但实际上，模型本身是不会记忆任何上下文的，只能依靠用户本身的输入去产生输出。而实现这个记忆功能，就需要额外的模块去保存我们和模型对话的上下文信息，然后在下一次请求时，把所有的历史信息都输入给模型，让模型输出最终结果。

而在 LangChain 中，提供这个功能的模块就称为 Memory，用于存储用户和模型交互的历史信息。在 LangChain 中根据功能和返回值的不同，会有多种不同的 Memory 类型，主要可以分为以下几个类别：

* 对话缓冲区内存（ConversationBufferMemory），最基础的内存模块，用于存储历史的信息
* 对话缓冲器窗口内存（ConversationBufferWindowMemory），只保存最后的 K 轮对话的信息，因此这种内存空间使用会相对较少
* 对话摘要内存（ConversationSummaryMemory），这种模式会对历史的所有信息进行抽取，生成摘要信息，然后将摘要信息作为历史信息进行保存。
* 对话摘要缓存内存（ConversationSummaryBufferMemory），这个和上面的作用基本一致，但是有最大 token 数的限制，达到这个最大 token 数的时候就会进行合并历史信息生成摘要

值得注意的是，对话摘要内存的设计出发点就是语言模型能支持的上下文长度是有限的（一般是 2048），超过了这个长度的数据天然的就被截断了。这个类会根据对话的轮次进行合并，默认值是 2，也就是每 2 轮就开启一次调用 LLM 去合并历史信息。

### 聊天消息历史 (ChatMessageHistory)

大多数（如果不是全部）内存模块的核心实用类之一是 ChatMessageHistory 类。这是一个超轻量级的包装器，它公开了方便的方法来保存人类消息、AI 消息，然后获取它们全部。
如果您在链外管理内存，可能需要直接使用此类。

In [31]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history.add_user_message("hi!")
history.add_ai_message("whats up?")

In [32]:
history.messages

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='whats up?', additional_kwargs={}, response_metadata={})]

### 对话缓冲区内存（ConversationBufferMemory）

让我们看看如何在链中使用这个模块，其中我们设置了verbose=True以便查看提示。

In [33]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation = ConversationChain(
    llm=model_ty, 
    verbose=True, 
    memory=ConversationBufferMemory()
)


/var/folders/fq/3pmzyx3s60l0sjm0lfcpdfhh0000gn/T/ipykernel_5091/542744488.py:7: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory()
/var/folders/fq/3pmzyx3s60l0sjm0lfcpdfhh0000gn/T/ipykernel_5091/542744488.py:4: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


In [34]:
conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


'Hello! How can I assist you today? If you have any questions or need information on a particular topic, feel free to ask!'

In [35]:
conversation.predict(input="你是谁")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hello! How can I assist you today? If you have any questions or need information on a particular topic, feel free to ask!
Human: 你是谁
AI:

> Finished chain.


'我是通义千问，阿里巴巴集团旗下的超大规模语言模型。我可以帮助你回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。如果你有任何问题或需要帮助，尽管告诉我，我会尽力提供支持！'

In [36]:
conversation.predict(input="你的背后原理是什么")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hello! How can I assist you today? If you have any questions or need information on a particular topic, feel free to ask!
Human: 你是谁
AI: 我是通义千问，阿里巴巴集团旗下的超大规模语言模型。我可以帮助你回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。如果你有任何问题或需要帮助，尽管告诉我，我会尽力提供支持！
Human: 你的背后原理是什么
AI:

> Finished chain.


'我的背后原理主要基于大规模的机器学习和深度学习技术。具体来说，我是通过以下几种关键技术实现的：\n\n1. **Transformer架构**：这是当前最先进的自然语言处理模型所采用的核心架构。Transformer能够高效地处理长文本序列，并且通过自注意力机制（self-attention mechanism）让模型更好地理解上下文关系。\n\n2. **大量数据训练**：我是在海量的互联网文本上进行训练的，这些数据涵盖了各种主题和领域，使得我能够理解和生成广泛话题的内容。\n\n3. **无监督与监督学习结合**：在预训练阶段，使用了无监督学习方法来学习语言的基础模式；而在微调阶段，则采用了监督学习的方式，以提高特定任务上的性能。\n\n4. **强化学习**：为了优化对话质量和用户体验，我还运用了来自人类反馈的强化学习技术，这有助于提升对话的自然度和连贯性。\n\n需要注意的是，虽然我可以解释自己的工作原理，但具体的算法细节和技术参数可能涉及敏感信息，因此不能完全公开。如果你对某个方面特别感兴趣，可以详细询问，我会尽量提供更多的信息。'

In [37]:
conversation.invoke("能简单点回答吗")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hello! How can I assist you today? If you have any questions or need information on a particular topic, feel free to ask!
Human: 你是谁
AI: 我是通义千问，阿里巴巴集团旗下的超大规模语言模型。我可以帮助你回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。如果你有任何问题或需要帮助，尽管告诉我，我会尽力提供支持！
Human: 你的背后原理是什么
AI: 我的背后原理主要基于大规模的机器学习和深度学习技术。具体来说，我是通过以下几种关键技术实现的：

1. **Transformer架构**：这是当前最先进的自然语言处理模型所采用的核心架构。Transformer能够高效地处理长文本序列，并且通过自注意力机制（self-attention mechanism）让模型更好地理解上下文关系。

2. **大量数据训练**：我是在海量的互联网文本上进行训练的，这些数据涵盖了各种主题和领域，使得我能够理解和生成广泛话题的内容。

3. **无监督与监督学习结合**：在预训练阶段，使用了无监督学习方法来学习语言的基础模式；而在微调阶段，则采用了监督学习的方式，以提高特定任务上的性能。

4. **强化学习**：为了优化对话质量和用户体验，我还运用了来自人类反馈的强化学习技术，这有助于提升对话的自然度和连贯性。

需要注意的是

{'input': '能简单点回答吗',
 'history': 'Human: Hi there!\nAI: Hello! How can I assist you today? If you have any questions or need information on a particular topic, feel free to ask!\nHuman: 你是谁\nAI: 我是通义千问，阿里巴巴集团旗下的超大规模语言模型。我可以帮助你回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。如果你有任何问题或需要帮助，尽管告诉我，我会尽力提供支持！\nHuman: 你的背后原理是什么\nAI: 我的背后原理主要基于大规模的机器学习和深度学习技术。具体来说，我是通过以下几种关键技术实现的：\n\n1. **Transformer架构**：这是当前最先进的自然语言处理模型所采用的核心架构。Transformer能够高效地处理长文本序列，并且通过自注意力机制（self-attention mechanism）让模型更好地理解上下文关系。\n\n2. **大量数据训练**：我是在海量的互联网文本上进行训练的，这些数据涵盖了各种主题和领域，使得我能够理解和生成广泛话题的内容。\n\n3. **无监督与监督学习结合**：在预训练阶段，使用了无监督学习方法来学习语言的基础模式；而在微调阶段，则采用了监督学习的方式，以提高特定任务上的性能。\n\n4. **强化学习**：为了优化对话质量和用户体验，我还运用了来自人类反馈的强化学习技术，这有助于提升对话的自然度和连贯性。\n\n需要注意的是，虽然我可以解释自己的工作原理，但具体的算法细节和技术参数可能涉及敏感信息，因此不能完全公开。如果你对某个方面特别感兴趣，可以详细询问，我会尽量提供更多的信息。',
 'response': '当然可以！简单来说，我是一个大型语言模型，基于阿里巴巴集团的研究成果。我的工作原理主要是通过学习大量的文本数据，理解人类的语言，并根据输入的信息生成相应的回答。我使用了一种叫做Transformer的先进技术，它帮助我更好地理解上下文和生成自然的回复。希望这个解释更清晰易懂！如果还有其他问题，随时问我哦。'}

In [38]:
conversation.memory.chat_memory.messages

[HumanMessage(content='Hi there!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello! How can I assist you today? If you have any questions or need information on a particular topic, feel free to ask!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='你是谁', additional_kwargs={}, response_metadata={}),
 AIMessage(content='我是通义千问，阿里巴巴集团旗下的超大规模语言模型。我可以帮助你回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。如果你有任何问题或需要帮助，尽管告诉我，我会尽力提供支持！', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='你的背后原理是什么', additional_kwargs={}, response_metadata={}),
 AIMessage(content='我的背后原理主要基于大规模的机器学习和深度学习技术。具体来说，我是通过以下几种关键技术实现的：\n\n1. **Transformer架构**：这是当前最先进的自然语言处理模型所采用的核心架构。Transformer能够高效地处理长文本序列，并且通过自注意力机制（self-attention mechanism）让模型更好地理解上下文关系。\n\n2. **大量数据训练**：我是在海量的互联网文本上进行训练的，这些数据涵盖了各种主题和领域，使得我能够理解和生成广泛话题的内容。\n\n3. **无监督与监督学习结合**：在预训练阶段，使用了无监督学习方法来学习语言的基础模式；而在微调阶段，则采用了监督学习的方式，以提高特定任务上的性能。\n\n4. **强化学习**：为了优化对话质量和用户体验，我还运用了来自人类反馈的强化学习技术，这有助于提升对话的

In [39]:
## 

### 保存消息历史

您经常需要保存消息，然后加载它们以便再次使用。这可以通过先将消息转换为普通的 Python 字典，保存这些字典（如 json 或其他格式），然后加载它们来轻松完成。以下是一个示例。

In [40]:
import json

from langchain.memory import ChatMessageHistory
from langchain.schema import messages_from_dict, messages_to_dict

In [41]:
conversation.memory.chat_memory

InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi there!', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! How can I assist you today? If you have any questions or need information on a particular topic, feel free to ask!', additional_kwargs={}, response_metadata={}), HumanMessage(content='你是谁', additional_kwargs={}, response_metadata={}), AIMessage(content='我是通义千问，阿里巴巴集团旗下的超大规模语言模型。我可以帮助你回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。如果你有任何问题或需要帮助，尽管告诉我，我会尽力提供支持！', additional_kwargs={}, response_metadata={}), HumanMessage(content='你的背后原理是什么', additional_kwargs={}, response_metadata={}), AIMessage(content='我的背后原理主要基于大规模的机器学习和深度学习技术。具体来说，我是通过以下几种关键技术实现的：\n\n1. **Transformer架构**：这是当前最先进的自然语言处理模型所采用的核心架构。Transformer能够高效地处理长文本序列，并且通过自注意力机制（self-attention mechanism）让模型更好地理解上下文关系。\n\n2. **大量数据训练**：我是在海量的互联网文本上进行训练的，这些数据涵盖了各种主题和领域，使得我能够理解和生成广泛话题的内容。\n\n3. **无监督与监督学习结合**：在预训练阶段，使用了无监督学习方法来学习语言的基础模式；而在微调阶段，则采用了监督学习的方式，以提高特定任务上的性能。\n\n4. **强化学习**：为了优化对话质量和用户

In [42]:
dicts = messages_to_dict(conversation.memory.chat_memory.messages)

In [43]:
dicts

[{'type': 'human',
  'data': {'content': 'Hi there!',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'human',
   'name': None,
   'id': None,
   'example': False}},
 {'type': 'ai',
  'data': {'content': 'Hello! How can I assist you today? If you have any questions or need information on a particular topic, feel free to ask!',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'ai',
   'name': None,
   'id': None,
   'example': False,
   'tool_calls': [],
   'invalid_tool_calls': [],
   'usage_metadata': None}},
 {'type': 'human',
  'data': {'content': '你是谁',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'human',
   'name': None,
   'id': None,
   'example': False}},
 {'type': 'ai',
  'data': {'content': '我是通义千问，阿里巴巴集团旗下的超大规模语言模型。我可以帮助你回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。如果你有任何问题或需要帮助，尽管告诉我，我会尽力提供支持！',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'ai',
   'name': None,
   'id': None,
   'example

In [44]:
import pickle

f = open("./memory", 'wb')
pickle.dump(dicts,f)
f.close()


In [45]:
dicts_load = pickle.load(open("./memory", "rb"))

In [46]:
#dicts_load

### 向对话添加记忆

https://stackoverflow.com/questions/75965605/how-to-persist-langchain-conversation-memory-save-and-load

In [47]:
new_messages = messages_from_dict(dicts_load)
retrieved_chat_history = ChatMessageHistory(messages=new_messages)
retrieved_memory = ConversationBufferMemory(chat_memory=retrieved_chat_history)

conversation_reload = ConversationChain(
    llm=model_ty, 
    verbose=True, 
    memory=retrieved_memory
)


In [48]:
conversation_reload.predict(input="我回来了")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hello! How can I assist you today? If you have any questions or need information on a particular topic, feel free to ask!
Human: 你是谁
AI: 我是通义千问，阿里巴巴集团旗下的超大规模语言模型。我可以帮助你回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。如果你有任何问题或需要帮助，尽管告诉我，我会尽力提供支持！
Human: 你的背后原理是什么
AI: 我的背后原理主要基于大规模的机器学习和深度学习技术。具体来说，我是通过以下几种关键技术实现的：

1. **Transformer架构**：这是当前最先进的自然语言处理模型所采用的核心架构。Transformer能够高效地处理长文本序列，并且通过自注意力机制（self-attention mechanism）让模型更好地理解上下文关系。

2. **大量数据训练**：我是在海量的互联网文本上进行训练的，这些数据涵盖了各种主题和领域，使得我能够理解和生成广泛话题的内容。

3. **无监督与监督学习结合**：在预训练阶段，使用了无监督学习方法来学习语言的基础模式；而在微调阶段，则采用了监督学习的方式，以提高特定任务上的性能。

4. **强化学习**：为了优化对话质量和用户体验，我还运用了来自人类反馈的强化学习技术，这有助于提升对话的自然度和连贯性。

需要注意的是

'欢迎回来！很高兴再次见到你。有什么新的问题或需要帮助的地方吗？无论是聊天、提问还是创作文字，我都在这里随时为你服务！ 😊'

### classwork 3

1. 请通过ConversationChain创建一个带记忆的对话，并使用该对话进行多次对话

2. 请将上述对话的上下文保存下来

3. 读取保存文件，请将上述对话的上下文加载到一个新的对话中